# **📄 Document type classification **

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 필요한 라이브러리를 설치합니다.

In [1]:
# 필요한 라이브러리 설치
!pip install timm
!pip install wandb
!pip install matplotlib
!pip install transformers

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [2]:
import os
import time
import random
import copy

import timm
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
import albumentations as A
from albumentations import ImageOnlyTransform
from augraphy import *
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

import wandb
import matplotlib.pyplot as plt

In [3]:
# 잘못된label 수정
train_df = pd.read_csv("../data/train.csv")
train_df.loc[428, 'target'] = 7
train_df.loc[1095, 'target'] = 14
train_df.loc[862, 'target'] = 3
train_df.loc[192, 'target'] = 7
train_df.loc[1237, 'target'] = 14
train_df.loc[38, 'target'] = 10
train_df.loc[340, 'target'] = 10

train_df.to_csv("../data/train.csv", index=False)

In [4]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [50]:
##중지1

# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None, oversample=False):
        self.df = pd.read_csv(csv)
        self.path = path
        self.transform = transform
        self.oversample = oversample

        if self.oversample:
            class_counts = np.bincount(self.df.values[:, 1].astype(int))

            max_class_count = max(class_counts)
            oversample_factors = [max_class_count // count for count in class_counts]
            oversample_factors[3] = 2
            oversample_factors[7] = 2 

            oversampled_data = [self.df.values[self.df.values[:, 1] == cls].repeat(factor, axis=0) for cls, factor in enumerate(oversample_factors)]
            oversampled_data = np.vstack(oversampled_data)

            self.df = pd.DataFrame(oversampled_data, columns=self.df.columns)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df.iloc[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert("RGB"))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target
    
meta_data = pd.read_csv('/data/ephemeral/home/data/meta.csv')
label_to_class_name = dict(zip(meta_data['target'], meta_data['class_name']))

In [53]:
class ImageDataset(Dataset):
    def __init__(self, csv_files, img_paths, transform=None):

        if isinstance(csv_files, str):
            csv_files = [csv_files]
        if isinstance(img_paths, str):
            img_paths = [img_paths]

        self.df = pd.concat([pd.read_csv(csv) for csv in csv_files], ignore_index=True)
        self.img_paths = img_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 데이터프레임에서 이미지 이름(ID)과 타겟 값을 가져오기
        img_name, target = self.df.iloc[idx]['ID'], self.df.iloc[idx]['target']
        img = None
        # target 값을 정수형으로 변환
        target = int(target)  # target 값을 정수형으로 변환합니다.

        # 이미지 경로 리스트를 순회하며 이미지 파일을 찾기
        for path in self.img_paths:
            img_path = os.path.join(path, img_name)
            if os.path.exists(img_path):
                img = Image.open(img_path).convert('RGB')
                # Move transform usage here
                inputs = self.transform(images=img, return_tensors="pt")
                input_pixels = inputs['pixel_values'].squeeze()  # Remove batch dimension
                break


        return input_pixels, torch.tensor(target, dtype=torch.long)


In [1]:
# one epoch 학습을 위한 함수
def training(model, dataloader, criterion, optimizer, device, epoch, num_epochs):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(dataloader)
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(images)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(labels.detach().cpu().numpy())

        pbar.set_description(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {loss.item()}")
        
    train_loss /= len(dataloader)
    train_acc = accuracy_score(targets_list, preds_list)    
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return model, train_loss, train_acc, train_f1

def evaluation(model, dataloader, criterion, device, epoch, num_epochs):
    model.eval()  # 모델을 평가 모드로 설정
    valid_loss = 0.0
    preds_list = []
    targets_list = []

    with torch.no_grad(): # model의 업데이트 막기
        tbar = tqdm(dataloader)
        for images, labels in tbar:
            images = images.to(device)
            labels = labels.to(device)

            preds = model(images)
            loss = criterion(preds, labels)

            valid_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(labels.detach().cpu().numpy())

            tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}] - Valid Loss: {loss.item()}")

    valid_loss = valid_loss / len(dataloader)
    valid_acc = accuracy_score(targets_list, preds_list)  
    valid_f1 = f1_score(targets_list, preds_list, average='macro')

    return valid_loss, valid_acc, valid_f1

def training_loop(model, train_dataloader, valid_dataloader, criterion, optimizer, device, num_epochs, patience, model_name, run):
    best_valid_loss = float('inf')  
    early_stop_counter = 0 
    valid_max_accuracy = -1
    best_model = None

    for epoch in range(num_epochs):
        model, train_loss, train_acc, train_f1 = training(model, train_dataloader, criterion, optimizer, device, epoch, num_epochs)
        valid_loss, valid_acc, valid_f1 = evaluation(model, valid_dataloader, criterion, device, epoch, num_epochs)

        monitoring_value = {'train_loss': train_loss, 'train_accuracy': train_acc, 'train_f1': train_f1, 
                            'valid_loss': valid_loss, 'valid_accuracy': valid_acc, 'valid_f1': valid_f1}
        
        run.log(monitoring_value, step=epoch)
        
        print(f'''Epoch [{epoch + 1}/{num_epochs}] Finished
        Train Loss: {train_loss}, Train Accuracy: {train_acc}, Train F1: {train_f1}
        Valid Loss: {valid_loss}, Valid Accuracy: {valid_acc}, Valid F1: {valid_f1}''')

        if valid_acc > valid_max_accuracy:
          valid_max_accuracy = valid_acc

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            best_model = model
            torch.save(model.state_dict(), f"./model_{model_name}.pt")
            early_stop_counter = 0
            print('Model Saved')

        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping")
            break

    return best_model, valid_max_accuracy

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [2]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '/data/ephemeral/home/data/'

# validation config
VALID_RATIO = 0.8

# model config
model_name = 'efficientnet_b4'
pretrained_size = 380
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds= [0.229, 0.224, 0.225]

# training config
LR = 5e-4
EPOCHS = 40
BATCH_SIZE = 32
dropout_ratio = 0.2
patience = 5
num_workers = 0
num_classes = 17



NameError: name 'torch' is not defined

In [58]:
import os

# 예를 들어, data_path를 절대 경로로 설정
data_path = "../data/"  # 실제 경로로 변경하세요

csv_list = [
    os.path.join(data_path, "A_train.csv"),
    os.path.join(data_path, "train.csv"),
    os.path.join(data_path, "A_13_train.csv"),
    os.path.join(data_path, "rotate_train.csv"),
    os.path.join(data_path, "rotateflip_train.csv"),
    os.path.join(data_path, "A_rotateandflip_train.csv")
]

path_list = [
    os.path.join(data_path, "augmented/A"),
    os.path.join(data_path, "train"),
    os.path.join(data_path, "augmented/A_13"),
    os.path.join(data_path, "augmented/rotate"),
    os.path.join(data_path, "augmented/rotateflip"),
    os.path.join(data_path, "augmented/rotateflip")
]

def check_files(files):
    for file in files:
        if not os.path.isfile(file):
            print(f"파일이 존재하지 않습니다: {file}")

def check_directories(directories):
    for directory in directories:
        if not os.path.isdir(directory):
            print(f"디렉토리가 존재하지 않습니다: {directory}")

check_files(csv_list)
check_directories(path_list)

import pandas as pd
from torch.utils.data import Dataset
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, csv_files, img_paths, transform=None):
        # CSV 파일 경로 확인
        for csv_file in csv_files:
            if not os.path.isfile(csv_file):
                raise FileNotFoundError(f"CSV 파일이 존재하지 않습니다: {csv_file}")
        
        # 모든 CSV 파일을 하나의 DataFrame으로 결합
        self.df = pd.concat([pd.read_csv(csv_file) for csv_file in csv_files], ignore_index=True)
        
        # 이미지 경로와 변환 함수 저장
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 이미지 경로와 레이블 가져오기
        img_name = os.path.join(self.img_paths[0], self.df.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.df.iloc[idx, 1]

        # 변환 함수가 제공되면 적용
        if self.transform:
            image = self.transform(image)

        return image, label

# 훈련 데이터셋 정의
trn_dataset = ImageDataset(
    csv_files=csv_list,
    img_paths=path_list,
    transform=train_transform
)

# 테스트 데이터셋 정의
tst_dataset = ImageDataset(
    csv_files=[os.path.join(data_path, "sample_submission.csv")],
    img_paths=[os.path.join(data_path, "test")],  # 단일 경로도 리스트로 감싸기
    transform=test_transform
)

print(len(trn_dataset), len(tst_dataset))



66655 3140


## 4. Load Data
* 학습, 검증, 테스트 데이터셋과 로더를 정의합니다.

In [59]:
# train image 변환을 위한 transform 코드
train_transform = A.Compose([
    # PatternGeneratorTransform(pattern, p=0.3), # 패턴 노이즈
    # A.Resize(height=pretrained_size, width=pretrained_size), # 이미지 크기 조정
    # 이미지 긴 측면 크기 조절 후 패딩 적용
    A.LongestMaxSize(max_size=pretrained_size, always_apply=True), 
    A.PadIfNeeded(min_height=pretrained_size, min_width=pretrained_size, border_mode=0, value=(255, 255, 255)),
    
    A.Normalize(mean=pretrained_means, std=pretrained_stds), # images normalization
    ToTensorV2() # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
])

# test image 변환을 위한 transform 코드
test_transform = A.Compose([    
    # A.Resize(height=pretrained_size, width=pretrained_size),
    A.LongestMaxSize(max_size=pretrained_size, always_apply=True),
    A.PadIfNeeded(min_height=pretrained_size, min_width=pretrained_size, border_mode=0, value=(255, 255, 255)),
    
    A.Normalize(mean=pretrained_means, std=pretrained_stds),
    ToTensorV2()
])

aug_test_transform = A.Compose([    
    A.RandomRotate90(),
    A.Flip(p=0.5),              
                        
    # A.Resize(height=pretrained_size, width=pretrained_size),
    A.LongestMaxSize(max_size=pretrained_size, always_apply=True),
    A.PadIfNeeded(min_height=pretrained_size, min_width=pretrained_size, border_mode=0, value=(255, 255, 255)),
    
    A.Normalize(mean=pretrained_means, std=pretrained_stds),
    ToTensorV2()
])

# 시각화를 위한 transform 코드
base_transform = A.Compose([
    ToTensorV2()
])

NameError: name 'img_size' is not defined

In [23]:

# Training Dataset 정의
train_dataset = ImageDataset(
    data_path + '../newtest/aug_train.csv',
    data_path + '../newtest/train/',
    transform=train_transform,
    oversample=True
)



# Test Dataset 정의
test_dataset = ImageDataset(
    data_path + '../newtest/sample_submission.csv',
    data_path + '../newtest/test/',
    transform=test_transform
)

aug_test_dataset = ImageDataset(
    data_path + '../newtest/sample_submission.csv',
    data_path + '../newtest/test/',
    transform=aug_test_transform
)

# 시각화용 Dataset 정의
train_dataset_v = ImageDataset(
    data_path + '../newtest/aug_train.csv',
    data_path + '../newtest/train/',
    transform=base_transform
)

test_dataset_v = ImageDataset(
    data_path + '../newtest/sample_submission.csv',
    data_path + '../newtest/test/',
    transform=base_transform
)

print(len(train_dataset), len(test_dataset))

14936 3140


In [ ]:
import os

data_path = "../data/"  

csv_list = [
    os.path.join(data_path, "A_train.csv"),
    os.path.join(data_path, "train.csv"),
    os.path.join(data_path, "A_13_train.csv"),
    os.path.join(data_path, "rotate_train.csv"),
    os.path.join(data_path, "rotateflip_train.csv"),
    os.path.join(data_path, "A_rotateandflip_train.csv")
]

path_list = [
    os.path.join(data_path, "augmented/A"),
    os.path.join(data_path, "train"),
    os.path.join(data_path, "augmented/A_13"),
    os.path.join(data_path, "augmented/rotate"),
    os.path.join(data_path, "augmented/rotateflip"),
    os.path.join(data_path, "augmented/rotateflip")
]

def check_files(files):
    for file in files:
        if not os.path.isfile(file):
            print(f"파일이 존재하지 않습니다: {file}")

def check_directories(directories):
    for directory in directories:
        if not os.path.isdir(directory):
            print(f"디렉토리가 존재하지 않습니다: {directory}")

check_files(csv_list)
check_directories(path_list)

import pandas as pd
from torch.utils.data import Dataset
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, csv_files, img_paths, transform=None):
        for csv_file in csv_files:
            if not os.path.isfile(csv_file):
                raise FileNotFoundError(f"CSV 파일이 존재하지 않습니다: {csv_file}")
        
        self.df = pd.concat([pd.read_csv(csv_file) for csv_file in csv_files], ignore_index=True)
        
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_paths[0], self.df.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.df.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

trn_dataset = ImageDataset(
    csv_files=csv_list,
    img_paths=path_list,
    transform=trn_transform
)

tst_dataset = ImageDataset(
    csv_files=[os.path.join(data_path, "sample_submission.csv")],
    img_paths=[os.path.join(data_path, "test")],  
    transform=tst_transform
)

print(len(trn_dataset), len(tst_dataset))



In [25]:
total_size = len(train_dataset)
train_num, valid_num = int(total_size * VALID_RATIO), total_size - int(total_size * VALID_RATIO)

generator = torch.Generator().manual_seed(SEED)
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_num, valid_num], generator = generator)

valid_data = copy.deepcopy(valid_dataset)
valid_data.dataset.transform = test_transform

print(f'Train dataset 개수: {len(train_dataset)}')
print(f'Validation dataset 개수: {len(valid_dataset)}')
print(f'Test dataset 개수: {len(test_dataset)}')

Train dataset 개수: 9558
Validation dataset 개수: 2390
Test dataset 개수: 3140


In [26]:
# DataLoader 정의
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
    )

valid_dataloader = DataLoader(
    valid_dataset, 
    batch_size = BATCH_SIZE, 
    shuffle = False,
    num_workers=0,
    pin_memory=True
    )

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
    )

aug_test_dataloader = DataLoader(
    aug_test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
    )

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [29]:
class AttentionModule(nn.Module):
    def __init__(self, in_features, out_features):
        super(AttentionModule, self).__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.Sigmoid()
        )

    def forward(self, x):
        attention_weights = self.attention(x)
        return x * attention_weights

class CustomEfficientNetB5(nn.Module):
    def __init__(self, num_classes, attention_size=1792):
        super(CustomEfficientNetB5, self).__init__()
        self.base_model = timm.create_model('efficientnet_b4', pretrained=True)
        
        # Remove the existing classifier
        self.base_model.reset_classifier(0, '')

        # Add attention mechanism
        self.attention = AttentionModule(attention_size, attention_size)

        # New classifier with attention
        self.classifier = nn.Linear(attention_size, num_classes)
        
    def forward(self, x):
        x = self.base_model(x)
        
        # Global average pooling
        x = x.mean([2, 3])

        # Apply attention mechanism
        x = self.attention(x)

        # Final classification
        x = self.classifier(x)

        return x

In [30]:
# 모델 생성
model = CustomEfficientNetB5(num_classes).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [31]:
model

CustomEfficientNetB4(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNormAct2d(
            48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, 

In [38]:
# # Access model weights
# model_weights = model.state_dict()

# # Print or visualize the weights
# for param_tensor in model_weights:
#     print(param_tensor, "\t", model_weights[param_tensor].size())

In [32]:
run = wandb.init(project = 'newwhy', name = 'training')

# wandb에 모델의 weight & bias, graident 시각화
# run.watch(model, loss_fn, log = 'all', log_graph = True)

model, valid_max_accuracy = training_loop(model, train_dataloader, valid_dataloader, loss_fn, optimizer, device, EPOCHS, patience, model_name, run)
print(f'Valid Max Accuracy: {valid_max_accuracy}')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: newwhy2 (newwhy2-korea-national-open-university). Use `wandb login --relogin` to force relogin


Epoch [1/40] - Valid Loss: 0.027063028886914253: 100%|██████████| 75/75 [00:16<00:00,  4.60it/s] 


Epoch [1/40] Finished
        Train Loss: 0.43505566125505146, Train Accuracy: 0.8559322033898306, Train F1: 0.8799261397206942
        Valid Loss: 0.05503783078553776, Valid Accuracy: 0.9811715481171548, Valid F1: 0.9859762747167916
Model Saved


Epoch [2/40] - Valid Loss: 0.000433526118285954: 100%|██████████| 75/75 [00:16<00:00,  4.64it/s]  


Epoch [2/40] Finished
        Train Loss: 0.03882033776900307, Train Accuracy: 0.9872358233940155, Train F1: 0.9908696068390567
        Valid Loss: 0.061180974115850405, Valid Accuracy: 0.9815899581589959, Valid F1: 0.9837733629386617


...

Epoch [6/40] Finished
        Train Loss: 0.01718978109538907, Train Accuracy: 0.9945595312826951, Train F1: 0.9950610940353255
        Valid Loss: 0.03390380808183788, Valid Accuracy: 0.992887029288703, Valid F1: 0.9952170719194524


Epoch [7/40] - Train Loss: 1.1351335160725284e-05: 100%|██████████| 299/299 [02:09<00:00,  2.31it/s]
Epoch [7/40] - Valid Loss: 8.276006701635197e-05: 100%|██████████| 75/75 [00:15<00:00,  4.80it/s] 


Epoch [7/40] Finished
        Train Loss: 0.01187925771046367, Train Accuracy: 0.9963381460556602, Train F1: 0.9973160522604942
        Valid Loss: 0.018454550314272636, Valid Accuracy: 0.99581589958159, Valid F1: 0.9970584924726782


Epoch [8/40] - Train Loss: 1.6093177919174195e-06: 100%|██████████| 299/299 [02:09<00:00,  2.32it/s]
Epoch [8/40] - Valid Loss: 5.3209873840387445e-06: 100%|██████████| 75/75 [00:15<00:00,  4.80it/s]

Epoch [8/40] Finished
        Train Loss: 0.006860943527633549, Train Accuracy: 0.9989537560159029, Train F1: 0.9994395058294597
        Valid Loss: 0.015029839516252348, Valid Accuracy: 0.9974895397489539, Valid F1: 0.9978844607493265
Early stopping
Valid Max Accuracy: 0.999163179916318


In [41]:
# run.unwatch()
run.finish()

train_accuracy,▁▇████████████
train_f1,▁▇████████████
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁
valid_accuracy,▅▅▁▆▃▇▄██▅█▇▆▇
valid_f1,▆▆▁▆▂▇▄▇█▆█▇▇▇
valid_loss,▄▄█▅▇▂▄▁▁▄▁▂▃▂
train_accuracy,0.99917
train_f1,0.99927
train_loss,0.00313
valid_accuracy,0.99946
valid_f1,0.99957


In [33]:
torch.cuda.empty_cache()

# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [35]:
model.load_state_dict(torch.load(f'./model_{model_name}.pt'))
model.to(device)

preds_list = []

model.eval()
for images, _ in tqdm(test_dataloader):
     images = images.to(device)

     with torch.no_grad():
         preds = model(images)
     preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:23<00:00,  4.27it/s]


In [36]:
model.load_state_dict(torch.load(f'./model_{model_name}.pt'))
model.to(device)

N_TTA = 20
preds_list = []
with torch.no_grad():
    loaders = [test_dataloader] + [aug_test_dataloader] * N_TTA

    for batches in tqdm(zip(*loaders), total=len(test_dataloader)):
        images, *aug_images = [images.to(device) for images, _ in batches]

        outputs_original = model(images)
        outputs_augmented = [model(aug_image) for aug_image in aug_images]

        final_outputs = (outputs_original + sum(outputs_augmented)) / N_TTA + 1
        preds_list.extend(final_outputs.argmax(dim=1).cpu().numpy())

# 예측 결과 확인
print("Ensemble Predictions:", preds_list)

100%|██████████| 99/99 [08:33<00:00,  5.19s/it]

Ensemble Predictions: [2, 12, 5, 0, 2, 13, 0, 8, 15, 11, 5, 7, 16, 9, 15, 4, 13, 5, 13, 12, 12, 12, 1, 6, 3, 0, 7, 16, 0, 6, 7, 0, 13, 2, 13, 16, 13, 14, 4, 0, 0, 9, 1, 9, 0, 13, 13, 0, 11, 14, 13, 10, 10, 6, 7, 12, 9, 5, 13, 13, 0, 13, 5, 8, 6, 1, 5, 7, 10, 6, 13, 10, 8, 15, 13, 15, 6, 1, 12, 13, 8, 9, 9, 13, 10, 10, 5, 13, 10, 0, 3, 8, 13, 15, 7, 16, 13, 11, 14, 13, 7, 7, 13, 0, 15, 11, 2, 13, 16, 8, 6, 2, 0, 13, 12, 16, 2, 7, 11, 14, 2, 13, 5, 8, 13, 12, 4, 4, 14, 6, 5, 13, 15, 0, 16, 16, 7, 6, 6, 13, 3, 8, 0, 2, 13, 8, 3, 13, 0, 0, 6, 8, 16, 12, 11, 16, 9, 15, 6, 8, 5, 5, 10, 10, 16, 3, 9, 12, 16, 5, 2, 8, 8, 16, 9, 13, 16, 16, 3, 4, 11, 15, 9, 9, 2, 3, 13, 10, 9, 0, 4, 0, 16, 5, 14, 13, 13, 13, 0, 1, 13, 2, 6, 16, 16, 13, 8, 9, 0, 10, 5, 0, 13, 0, 11, 2, 0, 4, 0, 13, 12, 13, 16, 7, 1, 5, 7, 0, 14, 6, 0, 3, 12, 13, 9, 0, 9, 10, 9, 13, 10, 14, 9, 11, 0, 0, 1, 11, 10, 6, 3, 3, 4, 7, 14, 15, 0, 13, 12, 4, 0, 15, 13, 13, 0, 12, 13, 4, 9, 0, 8, 3, 4, 5, 0, 0, 4, 0, 9, 12, 1, 7, 7, 13, 0

In [37]:
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [38]:
sample_submission_df = pd.read_csv(data_path + 'sample_submission.csv')
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [39]:
pred_df.to_csv("pred.csv", index=False)

In [40]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,0
4,00901f504008d884.jpg,2


- 리더보드 기준 최상위 예측값 앙상블